In [18]:
import pandas as pd
import numpy as np
import pickle
import csv
import time
import os
os.chdir('..')
from qmc import QMC
os.chdir('./data')

Load data in the format used by k. Train.

In [19]:
d = pd.read_csv('catsup_trainformat.csv', delimiter=',')
d.head(10)

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1
5,0,1,1,0,0,4.3,1
6,1,1,2,0,0,5.2,1
7,0,1,3,0,0,4.4,1
8,0,1,0,0,0,4.6,2
9,0,1,1,0,1,2.5,2


In [20]:
X = d.values[:,1:-1]
X.shape

(11192, 5)

In [43]:
def utilities(X, beta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    try:
        assert(X.shape == (11192, 5) and beta.shape == (3,300))
    except AssertionError:
        raise AssertionError('Ga X ff in juiste format gooien. X: %s, beta: %s' %(X.shape,beta.shape))
    beta_choice = np.zeros((3,11192))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i] = beta[:,id-1]
    eps = np.random.gumbel(scale=1,size=(11192,1))
    P = (X[:,2:]@ beta_choice)[:,0] +eps

#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = []
    for i in range(0,len(P), 4):
        choice = np.argmax(P[i:i+4])
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
        Y.append(int(choice))
    return np.array(Y)

In [22]:
#display_mean, feature_mean, price_mean, display_sigma, feature_sigma, price_disgma
theta_true = np.array([1.5,1,-1.1,0.8,0.1,1.2])
N = X.shape[0]
K = 3

In [24]:
draws = QMC(300,3,500)
pickle.dump( draws, open( str(500)+"_draws.p", "wb" ) )

In [49]:
%%time
def dgp(X: np.ndarray, D, dist = np.random.rand, draws = None):
    #X: dataset
    #D: amount of datasets
    np.random.seed(10)
    theta = np.array([1.5,1.,-1.1,0.8,0.1,1.2])
    the_big_dict = {}
    Y_array = np.zeros((2798,D))
    if np.any(draws):
        delta = draws
        method = 'QMC'
        print()
    else:
        delta = dist(300,3,D)
        method = 'MC'
    print(method)
    for i in range(D):
        delta_d = delta[:,:,i].T
        beta = theta[:K][:, None] + delta_d * theta[K:][:, None] ** 2
        Y = utilities(X,beta)
        Y_array[:,i] = Y
    the_big_dict['theta: %s'%(theta)] = Y_array
    return the_big_dict, method
    

Wall time: 0 ns


In [50]:
D = 500
dicter, method = dgp(X,D,draws=draws)
pickle.dump( dicter, open( "%i_%s_dgp.p"%(D,method), "wb" ) )


QMC


In [39]:
for key in dicter.keys():
    [print(dicter[key][i]) for i in range(30)]

[3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3.
 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3.
 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3.
 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 2. 2. 3. 3. 3. 3. 2. 3.
 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 2. 2. 3. 3. 3. 3. 2.
 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3.
 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3.
 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 2. 2. 3. 3.
 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 2. 2. 3.
 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 2. 2.
 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3.
 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3.
 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3.
 3. 2. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3. 2. 3. 3. 3. 3.

In [44]:
np.savetxt('dicter.csv', dicter['theta: [ 1.5  1.  -1.1  0.8  0.1  1.2]'], delimiter=',')

In [7]:
dicter = dgp(X,1)
for key in dicter.keys():
    [print(dicter[key][i]) for i in range(30)]

[2.]
[2.]
[3.]
[2.]
[2.]
[0.]
[3.]
[2.]
[0.]
[3.]
[2.]
[0.]
[2.]
[2.]
[2.]
[2.]
[2.]
[0.]
[0.]
[0.]
[0.]
[2.]
[2.]
[2.]
[2.]
[2.]
[1.]
[0.]
[0.]
[2.]
